In [1]:
import requests
import hmac
import hashlib
import base64
import time
import random
import re
import json
import math

appid = "1258909517"
bucket = " " #参考本文开头提供的链接
secret_id = "AKID6yb0uwpLsBML5mCZfWO2Yu12tVr4PqSq"  #参考官方文档
secret_key = "LTMzRhjY8UihS0JNDn9F7kCPN6aomVyE"  #同上
expired = time.time() + 2592000
onceExpired = 0
current = time.time()
rdm = ''.join(random.choice("0123456789") for i in range(10))
userid = "0"
fileid = "tencentyunSignTest"

info = "a=" + appid + "&b=" + bucket + "&k=" + secret_id + "&e=" + str(expired) + "&t=" + str(current) + "&r=" + str(rdm) + "&u=0&f="

signindex = hmac.new(bytes(secret_key,'utf-8'),bytes(info,'utf-8'), hashlib.sha1).digest()  # HMAC-SHA1加密
sign = base64.b64encode(signindex + bytes(info,'utf-8'))  # base64转码，也可以用下面那行转码
#sign=base64.b64encode(signindex+info.encode('utf-8'))

url = "https://recognition.image.myqcloud.com/ocr/handwriting"
headers = {'Host': 'recognition.image.myqcloud.com',
           "Authorization": sign,
           }
# files = {'appid': (None,appid),
#     'bucket': (None,bucket),
#     'image': ('1.jpg',open('D:/codes/images/form.jpg','rb'),'image/jpeg')
#     }     

# r = requests.post(url, files=files,headers=headers)
# responseinfo = r.content
# data = responseinfo.decode('utf-8')



In [2]:
import glob as gb

str='C:\\Users\\jjp\\Desktop\\out\\*.jpg'
img_path = gb.glob(str)
data_list=[]
for path in img_path:
    files = {'appid': (None,appid),
    'bucket': (None,bucket),
    'image': (path.split("\\")[-1]+'.jpg',open(path,'rb'),'image/jpeg')
    } 
    r = requests.post(url, files=files,headers=headers)
    responseinfo = r.content
    data = responseinfo.decode('utf-8')
    data_list.append(json.loads(data))
    
    

In [3]:
data_ok=filter(lambda x:x['message']=='OK',data_list)


In [4]:
def destance(rec1,rec2):
    d_x=(rec1['x']-rec2['x'])
    d_y=(rec1['y']-rec2['y'])
    distance = math.sqrt(d_x**2 + d_y**2)
    return distance

def find_text_group(json_data):
    low_dis=150 #最小间距
    items=json_data['data']['items']
    new_item=[]
    print(len(items))

    # new_item[new_item_index].append([items[0]['itemstring'],items[0]['itemcoord']])
    # items.sort(key=lambda x:x['itemcoord']['x']**2+x['itemcoord']['y']**2)
    for item in items: 
        out=filter(lambda x :destance(x['itemcoord'],item['itemcoord'])<low_dis,items)#过滤所有与元素相对距离＜low_dis的元素。
    #     new_item.append([[x['itemstring'],x['itemcoord']] for x in list(out)])
    #     list(out).sort(key=lambda x:destance(x['itemcoord'],item['itemcoord']))
        temp_list=[]
        for x in list(out):
            temp_list.append(items.pop(items.index(x)))#将元素分类进不同列表项内
#         print(str(len(items))+' last')
        new_item.append(temp_list)        
    #     out2=map(lambda x :[destance(x['itemcoord'],item['itemcoord']),items.index(x)] ,items)
    new_item2=[]
    for item in new_item:
        new_item2.append([[x['itemstring'],x['itemcoord']] for x in item])
    final_list=[]
    for item in new_item2:
    #     coor=[item[0][1]['x'],item[0][1]['y'],max([x[1]['width'] for x in item]),sum([x[1]['height'] for x in item])]
        coor=[item[0][1]['y'],item[0][1]['x'],item[-1][1]["y"]+item[-1][1]["height"],item[0][1]['x']+max([x[1]['width'] for x in item])]
        final_list.append(["".join([x[0] for x in item]),coor])
    print(final_list)
        
    return final_list
    


In [5]:
import win32com.client

app = win32com.client.Dispatch('Indesign.Application')


In [6]:
for data in list(data_ok):
    page=app.ActiveDocument.Pages.Add()
    print("page added")
    for item in find_text_group(data):

        my_text_frame=page.TextFrames.Add()
        my_text_frame.GeometricBounds=[float(i)*0.297 for i in item[1]]
        my_text_frame.Contents=item[0]
print(list(data_ok))

page added
23
[['四暗影毳笕司驫隔骂8TR&CA1132', [33, 51, 88, 174]], ['90G9712人学舞蹈的日富S', [55, 606, 157, 836]], ['看笔视频里姜本功超对的心^柯子我好生羡慕。我不禁蚨趺欲学着视预里的动作可我还是高住了成L', [187, 705, 285, 969]], ['箢~周第一唐其是这个动作 太难了', [381, 556, 544, 703]], ['XC黄艺嘉', [382, 141, 454, 229]], ['行感可表示宏奈T论我正样努力阻民就是上不去', [461, 715, 622, 827]]]
page added
47
[['3现在将是我太阳么山称霸的时丹', [43, 976, 171, 1082]], ['羅矚廄A9/13入自自口', [52, 8, 266, 163]], ['夏天李了树上的蝉也汗仍觉厚执才信地叫着。大阳义羰顽地{挂在天空，烤着大地。小花似已经破远得真不起匮啦._.草席', [116, 559, 241, 784]], ['我也不知道我为什不喜欢数学这个舟东。反正不论怎样认真听进我就是找不到一个可以让我要上它有油。原奋土约', [142, 227, 304, 439]], ['我望着太胆儿些想好扰啊c既好了敏望望天空温望着能客个规疗系。', [219, 817, 316, 989]], ['数学是我永远填不满的境是-怔过告的坎是个拔才出的萝日好于我这秤药学楂来说数回双3y', [365, 183, 501, 358]], ['我尽量止自己感觉凉决占。这不我躺在草席上吹着电风扇可依快感到保现我开启了乙静自残矛模式奥呼吸', [467, 573, 627, 773]], ['学最又栝噪又无趣我才相位也A尔说的什么数学充满|了趣味，哩辛好暑假有提前做级的练习这些作业才称得二*简单', [509, 38, 646, 254]], ['土区业老市容图静自傲京', [392, 826, 538, 906]]]
page added
41
[['四迎CUAP口49陌我弟鹈在辉的时候足以周藏滁德三恃', [39, 60, 197, 198]], ['5', [32, 980, 53, 995]], ['中这局车车我们注,值勤。我负责值勤L工亭这一块地万。幸好羊好

page added
42
[['10122', [61, 543, 76, 584]], ['一个顾容，去麻店天东西她招商品放到收银台上,收银员告诉独一共地块。她何十一块可不可以，收银负过容 十三块五读给您', [72, 754, 196, 974]], ['ALrrg32101 21剪双一根长长的细线,两根牙签做了一日个筒陋幼传括筒”。首份拿牙签在材子===', [66, 16, 213, 301]], ['民部据~傲心，癌把牙签插进去。用线绑雕雅牙签就不容易从杯子里胎落这个练称接毒另一个杯子我一手拿看线一手握着剪|风一会儿穿线一会绑线', [176, 171, 379, 423]], ['f一块，就+-决。不行这已经这最低析了。*哎呀+-块卖给我叹“不卖我就上别的店看看叶行行行卖给你。”十二块给你。', [216, 817, 433, 1003]], ['那就十二决，行吧7”加一浅。”行行好吧姐姐我是小学生很穷的。“十三块,不能再少了。’开心工', [359, 539, 555, 737]], ['嘻嘻效尧还真不错1我着称子将事何愈段线四值朝着杯嘿轻轻地圆听程到近得北7弟弟在那头拼命地点头。', [501, 159, 640, 357]]]
page added
30
[['3义', [45, 956, 128, 998]], ['咱打班就有个同字患上了严重的健忘症，晨，他机足塔着铃芦进教室老国师电专他问红领巾一匀培还没有境和他就触电似的往下随。', [69, 292, 235, 474]], ['10123我们班里有许多同学都得了健己扉海马虑病唉这两种病由赖而生泌须自己康复不用吃', [95, 30, 259, 177]], ['呀，我了。"他如梦初烧本子给我。老师提醒道。噢’他开抱里里外外地我上下下地找。', [240, 286, 415, 448]], ['他蹲下身在台肚里我来找去。时间一分秒地过去。他站起来一脸 迷芒地望寒老师。', [318, 367, 495, 493]]]
page added
32
[['11载题照醒费蹊品OAX236', [38, 59, 108, 184]], ['37我客好了校服,在镜子前了文蜒扭望', [49, 955, 144, 1099]], ['10125，早上醒来哉揉了探眼睛欣并破子-个鳄昼打艇”', [59, 

page added
45
[['1陈药A68大家的纸都交上去了 敫动人儿', [39, 59, 146, 173]], ['19', [44, 973, 66, 988]], ['规则是这科约分数最富的几位胜任中队长其怜的8位小队长.中队长还剩最后名额。18↑聚数呙的同学联任。34霁人下的就只能当', [86, 551, 278, 724]], ['的时刻到来了一一唱票1如果~个底绩好的人。是用才着担儿的，因为他有百分工八十五的极率胜住 。而我捫进不全优都没有。', [151, 168, 329, 434]], ['老百姓穿就差那4一口气别走你是我画', [178, 811, 386, 959]], ['我紧张得手脚冰岗，盯转睛地十江着黑板。o上, *XXX 3, Xxx，25阔学喝票胡老师北專数号到絮核上。', [268, 32, 447, 212]], ['惊队长了疗想当中队长,做梦吧你1', [285, 532, 395, 645]], ['我巧票。栽昨关勋8经预料到我当小队长{是及呵题的可中队长票数肯定不够。', [504, 42, 604, 206]], ['有的同等落二了,儿建很难过记得在以前，我也落造过', [494, 754, 592, 865]]]
page added
33
[['71', [48, 967, 69, 988]], ['1/123日品今天我在厕所里碰制天与。她~见我就境“李天切均征学校带了好多的', [61, 543, 172, 735]], ['回)5', [74, 217, 130, 255]], ['70回家后,我把哦继续白小队长的局息一共铂了妈妈。她调|继续保特也是一种胜种', [90, 3, 249, 149]], ['水宝宝，铅笔袋望都装洞了。出均均的保桌附近还散落着许多未家大的月小的水堂宝。', [178, 755, 379, 948]], ['~当 葫考师发现启她非常生气当即就把水宝呈报进了垃圾植e听胡老师谠这水宝', [294, 651, 470, 797]], ['里是很脏的，抓过2后都得去先手据我所知这东西该是有我的。I', [478, 530, 549, 702]]]
page added
40
[['7', [45, 954, 64, 981]], ['72112今天,我和弟弟爸卜爸一起啮了搭木块。把小

page added
31
[['品，A等104留', [66, 19, 146, 120]], ['105', [72, 951, 89, 981]], ['12/ {输甸', [63, 537, 152, 592]], ['我改，我t改我改改改太好3!看不出来轻松)', [125, 375, 316, 456]], ['援下来副见个恃胡老师在己前的作止和练习$I都证我们练过.细细瑞坪日', [279, 50, 541, 164]], ['吕口哈哦昨是我看了看棋子的位置，发瓒↑想这一排只有三颗。才要最这里已经有地颗引得快产堵我经过一香承思熟', [370, 793, 578, 961]]]
page added
35
[['173s药106韩井', [37, 61, 163, 139]], ['12/ 4心必口', [60, 530, 207, 580]], ['=舅要塔 婚吨!家果到处娃着西着之球桂潑都是太红的嬉气,羊羊的。', [123, 778, 228, 947]], ['爸爹笑孤声这!预苏看他找到四颗棋的了!我宝忙茬棋盘上找来堵上。呀“在这可是我堵是我太惨太意了1', [125, 219, 328, 348]], ['s', [234, 532, 253, 545]], ['找去把石了区五他熟下在那头，我堵那头他就下在汉义', [162, 367, 352, 476]], ['唉为了这婚礼珍请过两犬假，s了好的声梁的重要内窃见两楠作建', [413, 562, 486, 725]]]
page added
32
[['109', [79, 965, 98, 993]], ['198', [115, 16, 133, 50]], ['啊?来了来了!不-会儿，新娘、即就下婚车一步一咬池朝刀口吱来。我三莎开作两步扬起盛子了三法a', [87, 668, 224, 862]], ['小姨交名我个任务在新娘走进行地包时我要徐地大上撒花新。且', [126, 253, 307, 388]], ['我去抓了一把花翔气往新娠天疏捷的再亭卓?再商阿', [322, 567, 417, 703]], ['来到口等新妮距离新妮郎来过剩下7分钟，我便琶苦起来花瓣征上抗还老狂下应读往上花瓣亭二该要从上面暮下来才好春1', [423, 154, 664, 349]], ['埃1垛?

In [7]:
# for index, item in enumerate(items):
# #     print (item)
# #     print (index)
# #     print (destance(item['itemcoord'],items[index+1]['itemcoord']))
#     if index<len(items)-1:
#         if (destance(item['itemcoord'],items[index+1]['itemcoord']))<low_dis:
#             new_item[new_item_index].append([items[index+1]['itemstring'],items[index+1]['itemcoord']])

#         else:
#             new_item.append([])
#             new_item_index+=1
#             new_item[new_item_index].append([items[index+1]['itemstring'],items[index+1]['itemcoord']])
# print(new_item_index)

# for index,item in enumerate(new_item):

#     try:    
#         for i in range(len(new_item)-index-1):
# #             print ((len(new_item)-index-1))
#             if (destance(item[-1][1],new_item[index+i+1][0][1]))<low_dis:
#                 item+=new_item[index+i+1]
#                 del(new_item[index+i+1])


#     except :
#         print("Error: ")